In [1]:
# load in required packages
import numpy as np
import pandas as pd
import geopandas as gpd
import shap
import re
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, cross_validate, GroupKFold, StratifiedGroupKFold, RepeatedKFold, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, linear_model
from xgboost import XGBClassifier, XGBRegressor
import statsmodels.formula.api as smf
from datetime import datetime
import matplotlib.pyplot as plt
import os
import pickle

cities_all=['Berlin','Dresden','Düsseldorf','Frankfurt am Main','Kassel','Leipzig','Magdeburg','Potsdam','Clermont','Dijon','Lille','Lyon','Montpellier','Nantes','Nimes','Paris','Toulouse','Madrid','Wien','France_other','Germany_other']
countries=['Germany','Germany','Germany','Germany','Germany','Germany','Germany','Germany','France','France','France','France','France','France','France','France','France','Spain','Austria','France','Germany']
#form_str="Mode_num ~ FeatureM_Trip_Time + FeatureM_Season + FeatureM_Trip_Purpose_Agg + FeatureM_Sex + FeatureM_Age + FeatureM_Trip_Distance + FeatureM_CarOwnershipHH + FeatureM_HHSize + FeatureM_Occupation +  FeatureM_Education  + FeatureM_UrbPopDensity_origin +  FeatureM_DistSubcenter_origin + FeatureM_DistCenter_origin +  FeatureM_UrbBuildDensity_origin +  FeatureM_IntersecDensity_origin +  FeatureM_street_length_origin +FeatureM_bike_lane_share_origin +  FeatureM_LU_UrbFab_origin + FeatureM_LU_Comm_origin + FeatureM_transit_accessibility_origin + FeatureM_Country" # FeatureM_CarAvailable replaced by FeatureM_CarOwnershipHH
form_str="Mode_num ~ FeatureM_Trip_Time + FeatureM_Season + FeatureM_Trip_Purpose_Agg + FeatureM_Sex + FeatureM_Age + FeatureM_Trip_Distance + FeatureM_CarOwnershipHH + FeatureM_HHSize + FeatureM_Occupation +  FeatureM_Education  + FeatureM_UrbPopDensity_res +  FeatureM_DistSubcenter_res + FeatureM_DistCenter_res_pc +  FeatureM_UrbBuildDensity_res +  FeatureM_IntersecDensity_res +  FeatureM_street_length_res +FeatureM_bike_lane_share_res +  FeatureM_LU_UrbFab_res + FeatureM_LU_Comm_res + FeatureM_transit_accessibility_res + FeatureM_Country" # FeatureM_CarAvailable replaced by FeatureM_CarOwnershipHH


pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [2]:
stat=pd.read_csv('../outputs/summary_stats/summary_UF_all_reweighted.csv')
d2c=stat.loc[:,['cities','Dist. to center']]
d2c.rename(columns={'cities':'City'},inplace=True)

In [3]:
city0='Berlin'
df0=pd.read_csv('../outputs/Combined/' + city0 + '_UF.csv')
print(len(df0.columns), 'columns in the data for ', city0)
df0['City']=city0
df0=df0.merge(d2c)
df0['DistCenter_res_pc']=100*df0['DistCenter_res']/df0['Dist. to center']
df_all=df0.copy()

cities=['Dresden','Leipzig','Magdeburg','Potsdam','Frankfurt am Main','Düsseldorf','Kassel']
for city1 in cities:
    print(city1)
    df1=pd.read_csv('../outputs/Combined/' + city1 + '_UF.csv')
    print(len(df1.columns), 'columns in the data for ', city1)
    df1['City']=city1
    df1=df1.merge(d2c)
    df1['DistCenter_res_pc']=100*df1['DistCenter_res']/df1['Dist. to center']
    if len(df1.columns==df_all.columns):
        df_all=pd.concat([df_all,df1])
        print(city1, 'added.')
        print(len(df_all), 'rows in the combined dataframe')
df_all['HHNR']=df_all['City']+'_'+df_all['HHNR'].astype(int).astype(str)
df_all['HH_PNR']=df_all['City']+'_'+df_all['HH_PNR'].astype(int).astype(str)
df_all['HH_P_WNR']=df_all['City']+'_'+df_all['HH_P_WNR'].astype(int).astype(str)
df_DE=df_all.copy()
df_DE['Start']=df_DE['trip_type_all'].str[:4]
df_DE=df_DE.loc[df_DE['Start']=='Home',]
df_DE['Country']='Germany'

df_DE=df_DE.loc[:,('City', 'HH_P_WNR','HH_PNR', 'HHNR','Ori_geocode', 'Des_geocode','Res_geocode',#'Dist_group', # IDs, trip geocodes, home-Res_geocode
    'Trip_Time', 'Season','Trip_Purpose_Agg','CarOwnershipHH', 'Trip_Distance', # trip details, keep number of accompanying householders now as 'TravelAlone', but remove n_others_car as it gives away the mode.     
    'HHSize', #'IncomeDetailed_Numeric', #'IncomeDetailed', 'HHType', # household details
    'Sex',  'Occupation', 'Education','Age', # 'MobilityConstraints',
    'UrbPopDensity_res','DistSubcenter_res', 'DistCenter_res_pc','UrbBuildDensity_res',# 'MeanTime2Transit_res',
    'IntersecDensity_res', 'street_length_res','bike_lane_share_res', 'transit_accessibility_res',# 'diff', 'K_avg_res', 'StreetDensity_res', 'StreetsPerNode_res', 'K_avg_dest','StreetDensity_dest', 'StreetsPerNode_dest', 
    'LU_UrbFab_res','LU_Comm_res','Country',# urban form features, land-use features are now all from UA. removed 'LU_Road_res', 'LU_Road_dest',
    # target: mode
    'Mode')
    ]

# France
city0='Clermont'
df0=pd.read_csv('../outputs/Combined/' + city0 + '_UF.csv')
df0.drop(columns=['IncomeDetailed', 'IncomeHarmonised','Des_Sec_Zone', 'Month', 'Ori_Sec_Zone','Res_Sec_Zone', 'Sample','Sector_Zone', 'Zone','geo_unit','N_Stops', 'N_Legs','Mode_disagg'],errors='ignore',inplace=True)
print(len(df0.columns), 'columns in the data for ', city0)
df0['City']=city0
df0=df0.merge(d2c)
df0['DistCenter_res_pc']=100*df0['DistCenter_res']/df0['Dist. to center']
df_all=df0.copy()

cities=['Toulouse','Montpellier','Lyon','Nantes','Nimes','Lille','Dijon']
for city1 in cities:
    print(city1)
    df1=pd.read_csv('../outputs/Combined/' + city1 + '_UF.csv')
    df1.drop(columns=['IncomeDetailed', 'IncomeHarmonised', 'Des_Sec_Zone', 'Month', 'Ori_Sec_Zone','Res_Sec_Zone',  'Sample','Sector_Zone', 'Zone','geo_unit',
                    'Commune', 'Des_Cell', 'Grid_Cell', 'NoMobilityConstraints','Ori_Cell','N_Stops', 'N_Legs','Mode_disagg'],errors='ignore',inplace=True) # plus spme non-shared Paris variables
    print(len(df1.columns), 'columns in the data for ', city1)
    df1['City']=city1
    df1=df1.merge(d2c)
    df1['DistCenter_res_pc']=100*df1['DistCenter_res']/df1['Dist. to center']
    if len(df1.columns==df_all.columns):
        df1=df1[df_all.columns] # this is required for Paris, where the same columns exist after the dropping above, but the order is different
        df_all=pd.concat([df_all,df1])
        print(city1, 'added.')
        print(len(df_all), 'rows in the combined dataframe')
df_all['HHNR']=df_all['City']+'_'+df_all['HHNR'].astype(str)
df_all['HH_PNR']=df_all['City']+'_'+df_all['HH_PNR'].astype(str)
df_all['HH_P_WNR']=df_all['City']+'_'+df_all['HH_P_WNR'].astype(str)
df_FR=df_all.copy()
df_FR['Start']=df_FR['trip_type_all'].str[:4]
df_FR=df_FR.loc[df_FR['Start']=='Home',]
df_FR['Country']='France'
df_FR=df_FR.loc[:,('City','HH_P_WNR','HH_PNR', 'HHNR','Ori_geocode', 'Des_geocode','Res_geocode',#'Dist_group', # IDs, trip geocodes, home-Res_geocode
    'Trip_Time', 'Season','Trip_Purpose_Agg','CarOwnershipHH', 'Trip_Distance', # trip details, keep number of accompanying householders now as 'TravelAlone', but remove n_others_car as it gives away the mode.     
    'HHSize', #'IncomeDetailed_Numeric', #'IncomeDetailed', 'HHType', # household details
    'Sex',  'Occupation', 'Education','Age', # 'MobilityConstraints',
    'UrbPopDensity_res','DistSubcenter_res', 'DistCenter_res_pc','UrbBuildDensity_res',# 'MeanTime2Transit_res',
    'IntersecDensity_res', 'street_length_res','bike_lane_share_res','transit_accessibility_res', # 'diff', 'K_avg_res', 'StreetDensity_res', 'StreetsPerNode_res', 'K_avg_dest','StreetDensity_dest', 'StreetsPerNode_dest', 
    'LU_UrbFab_res','LU_Comm_res', 'Country',  # urban form features, land-use features are now all from UA. removed 'LU_Road_res', 'LU_Road_dest',
    # target: mode
    'Mode')
    ]
city='Paris'
df=pd.read_csv('../outputs/Combined/' + city + '_UF.csv')
df['City']=city
df=df.merge(d2c)
df['DistCenter_res_pc']=100*df['DistCenter_res']/df['Dist. to center']
df['Country']='France'
df['HHNR']=df['City']+'_'+df['HHNR'].astype(str)
df['HH_PNR']=df['City']+'_'+df['HH_PNR'].astype(str)
df['HH_P_WNR']=df['City']+'_'+df['HH_P_WNR'].astype(str)
df['Start']=df['trip_type_all'].str[:4]
df=df.loc[df['Start']=='Home',]

df_Paris=df.copy()
df_Paris=df_Paris.loc[:,('City','HH_P_WNR','HH_PNR', 'HHNR','Ori_geocode', 'Des_geocode','Res_geocode',#'Dist_group', # IDs, trip geocodes, home-Res_geocode
    'Trip_Time', 'Season','Trip_Purpose_Agg','CarOwnershipHH', 'Trip_Distance', # trip details, keep number of accompanying householders now as 'TravelAlone', but remove n_others_car as it gives away the mode.     
    'HHSize', #'IncomeDetailed_Numeric', #'IncomeDetailed', 'HHType', # household details
    'Sex',  'Occupation', 'Education','Age', # 'MobilityConstraints',
    'UrbPopDensity_res','DistSubcenter_res', 'DistCenter_res_pc','UrbBuildDensity_res',# 'MeanTime2Transit_res',
    'IntersecDensity_res', 'street_length_res','bike_lane_share_res','transit_accessibility_res', # 'diff', 'K_avg_res', 'StreetDensity_res', 'StreetsPerNode_res', 'K_avg_dest','StreetDensity_dest', 'StreetsPerNode_dest', 
    'LU_UrbFab_res','LU_Comm_res', 'Country',  # urban form features, land-use features are now all from UA. removed 'LU_Road_res', 'LU_Road_dest',
    # target: mode
    'Mode')
    ]

city='Madrid'
df=pd.read_csv('../outputs/Combined/' + city + '_UF.csv')
df['City']=city
df=df.merge(d2c)
df['DistCenter_res_pc']=100*df['DistCenter_res']/df['Dist. to center']
df['Country']='Spain'
df['HHNR']=df['City']+'_'+df['HHNR'].astype(str)
df['HH_PNR']=df['City']+'_'+df['HH_PNR'].astype(str)
df['HH_P_WNR']=df['City']+'_'+df['HH_P_WNR'].astype(str)
df['Start']=df['trip_type_all'].str[:4]
df=df.loc[df['Start']=='Home',]

df_Madrid=df.copy()
df_Madrid=df_Madrid.loc[:,('City','HH_P_WNR','HH_PNR', 'HHNR','Ori_geocode', 'Des_geocode','Res_geocode',#'Dist_group', # IDs, trip geocodes, home-Res_geocode
    'Trip_Time', 'Season','Trip_Purpose_Agg','CarOwnershipHH', 'Trip_Distance', # trip details, keep number of accompanying householders now as 'TravelAlone', but remove n_others_car as it gives away the mode.     
    'HHSize', #'IncomeDetailed_Numeric', #'IncomeDetailed', 'HHType', # household details
    'Sex',  'Occupation', 'Education','Age', # 'MobilityConstraints',
    'UrbPopDensity_res','DistSubcenter_res', 'DistCenter_res_pc','UrbBuildDensity_res',# 'MeanTime2Transit_res',
    'IntersecDensity_res', 'street_length_res','bike_lane_share_res','transit_accessibility_res', # 'diff', 'K_avg_res', 'StreetDensity_res', 'StreetsPerNode_res', 'K_avg_dest','StreetDensity_dest', 'StreetsPerNode_dest', 
    'LU_UrbFab_res','LU_Comm_res','Country',   # urban form features, land-use features are now all from UA. removed 'LU_Road_res', 'LU_Road_dest',
    # target: mode
    'Mode')
    ]

city='Wien'
df=pd.read_csv('../outputs/Combined/' + city + '_UF.csv')
df['City']=city
df=df.merge(d2c)
df['DistCenter_res_pc']=100*df['DistCenter_res']/df['Dist. to center']
df['Country']='Austria'
df['HHNR']=df['City']+'_'+df['HHNR'].astype(str)
df['HH_PNR']=df['City']+'_'+df['HH_PNR'].astype(str)
df['HH_P_WNR']=df['City']+'_'+df['HH_P_WNR'].astype(str)
df['Start']=df['trip_type_all'].str[:4]
df=df.loc[df['Start']=='Home',]

df_Wien=df.copy()
df_Wien=df_Wien.loc[:,('City','HH_P_WNR','HH_PNR', 'HHNR','Ori_geocode', 'Des_geocode','Res_geocode',#'Dist_group', # IDs, trip geocodes, home-Res_geocode
    'Trip_Time', 'Season','Trip_Purpose_Agg','CarOwnershipHH', 'Trip_Distance', # trip details, keep number of accompanying householders now as 'TravelAlone', but remove n_others_car as it gives away the mode.     
    'HHSize', #'IncomeDetailed_Numeric', #'IncomeDetailed', 'HHType', # household details
    'Sex',  'Occupation', 'Education','Age', # 'MobilityConstraints',
    'UrbPopDensity_res','DistSubcenter_res', 'DistCenter_res_pc','UrbBuildDensity_res',# 'MeanTime2Transit_res',
    'IntersecDensity_res', 'street_length_res','bike_lane_share_res','transit_accessibility_res', # 'diff', 'K_avg_res', 'StreetDensity_res', 'StreetsPerNode_res', 'K_avg_dest','StreetDensity_dest', 'StreetsPerNode_dest', 
    'LU_UrbFab_res','LU_Comm_res','Country',   # urban form features, land-use features are now all from UA. removed 'LU_Road_res', 'LU_Road_dest',
    # target: mode
    'Mode')
    ]

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.


77 columns in the data for  Berlin
Dresden
77 columns in the data for  Dresden
Dresden added.
126573 rows in the combined dataframe
Leipzig
77 columns in the data for  Leipzig
Leipzig added.
138169 rows in the combined dataframe
Magdeburg
77 columns in the data for  Magdeburg
Magdeburg added.
147545 rows in the combined dataframe
Potsdam
77 columns in the data for  Potsdam
Potsdam added.
153368 rows in the combined dataframe
Frankfurt am Main
77 columns in the data for  Frankfurt am Main
Frankfurt am Main added.
159403 rows in the combined dataframe
Düsseldorf
77 columns in the data for  Düsseldorf
Düsseldorf added.
183247 rows in the combined dataframe
Kassel
77 columns in the data for  Kassel
Kassel added.
192614 rows in the combined dataframe


Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.


58 columns in the data for  Clermont
Toulouse
58 columns in the data for  Toulouse
Toulouse added.
49864 rows in the combined dataframe
Montpellier
58 columns in the data for  Montpellier
Montpellier added.
80127 rows in the combined dataframe
Lyon
58 columns in the data for  Lyon
Lyon added.
132884 rows in the combined dataframe
Nantes
58 columns in the data for  Nantes
Nantes added.
167926 rows in the combined dataframe
Nimes
58 columns in the data for  Nimes
Nimes added.
179143 rows in the combined dataframe
Lille
58 columns in the data for  Lille
Lille added.
215992 rows in the combined dataframe
Dijon
58 columns in the data for  Dijon
Dijon added.
230246 rows in the combined dataframe


Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
df=pd.concat([df_DE,df_FR,df_Paris,df_Madrid,df_Wien],ignore_index=True)

Occ_dict={'Employed_FullTime':'Employed','Employed_PartTime':'Employed','Employed':'Employed','Trainee':'Employed',
          'Student_School':'Student_School','Student_3rdLevel':'Student_3rdLevel','Pre-School':'Pre-School','Retired':'Retired',
          'Unemployed':'Unemployed/Other','Other':'Unemployed/Other','Home_Partner':'Unemployed/Other'}
Edu_dict={'University':'University','Secondary':'Secondary','Secondary+BAC':'Secondary','Secondary+Matura':'Secondary',
          'Apprenticeship':'Apprenticeship',
          'Elementary':'Primary/None','Pre-School':'Primary/None','No diploma yet':'Primary/None','Unknown':'Primary/None','Other':'Primary/None'}

df['Occupation']=df['Occupation'].map(Occ_dict)
df['Education']=df['Education'].map(Edu_dict)

df

,City,HH_P_WNR,HH_PNR,HHNR,Ori_geocode,Des_geocode,Res_geocode,Trip_Time,Season,Trip_Purpose_Agg,...,DistCenter_res_pc,UrbBuildDensity_res,IntersecDensity_res,street_length_res,bike_lane_share_res,transit_accessibility_res,LU_UrbFab_res,LU_Comm_res,Country,Mode
0,Berlin,Berlin_10295611,Berlin_1029561,Berlin_102956,10115,10115,10115,Daytime Off-Peak,Spring,Home↔Work,...,23.413879,9.010181e+06,39.380867,127.710059,0.0965,9.428305e+01,0.395459,0.366144,Germany,Transit
1,Berlin,Berlin_10296411,Berlin_1029641,Berlin_102964,10115,10245,10115,AM_Rush,Winter,Home↔Work,...,23.413879,9.010181e+06,39.380867,127.710059,0.0965,9.428305e+01,0.395459,0.366144,Germany,Bike
2,Berlin,Berlin_10296421,Berlin_1029642,Berlin_102964,10115,10178,10115,Daytime Off-Peak,Winter,Home↔Work,...,23.413879,9.010181e+06,39.380867,127.710059,0.0965,9.428305e+01,0.395459,0.366144,Germany,Bike
3,Berlin,Berlin_10296431,Berlin_1029643,Berlin_102964,10115,10119,10115,AM_Rush,Winter,Home↔School,...,23.413879,9.010181e+06,39.380867,127.710059,0.0965,9.428305e+01,0.395459,0.366144,Germany,Bike
4,Berlin,Berlin_10296433,Berlin_1029643,Berlin_102964,10115,10179,10115,PM Rush,Winter,Home↔Leisure,...,23.413879,9.010181e+06,39.380867,127.710059,0.0965,9.428305e+01,0.395459,0.366144,Germany,Transit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277493,Wien,Wien_9066_4_1,Wien_9066_4,Wien_9066,31701,91701,31701,AM_Rush,Autumn,Home↔School,...,247.553868,4.250415e+03,6.937562,135.293102,0.0000,4.361560e-13,0.038474,0.023820,Austria,Transit
277494,Wien,Wien_15603_1_1,Wien_15603_1,Wien_15603,31701,30735,31701,AM_Rush,Autumn,Home↔Shopping,...,247.553868,4.250415e+03,6.937562,135.293102,0.0000,4.361560e-13,0.038474,0.023820,Austria,Car
277495,Wien,Wien_15603_2_1,Wien_15603_2,Wien_15603,31701,30735,31701,AM_Rush,Autumn,Home↔Shopping,...,247.553868,4.250415e+03,6.937562,135.293102,0.0000,4.361560e-13,0.038474,0.023820,Austria,Car
277496,Wien,Wien_10149_1_1,Wien_10149_1,Wien_10149,31711,31723,31711,AM_Rush,Autumn,Home↔Companion,...,187.264157,2.350708e+03,23.465284,132.376026,0.0113,1.554669e-02,0.090456,0.039216,Austria,Car


In [5]:
df['Mode_num']=0
# car, bike, foot, transit
df.loc[df['Mode']=='Car','Mode_num']=0
df.loc[df['Mode']=='Bike','Mode_num']=1
df.loc[df['Mode']=='Foot','Mode_num']=2
df.loc[df['Mode']=='Transit','Mode_num']=3

df['UrbPopDensity_res']=0.01*df['UrbPopDensity_res'] # convert from per/km2 to per/ha
df.loc[:,['bike_lane_share_res','LU_UrbFab_res','LU_Comm_res']]=100*df.loc[:,['bike_lane_share_res','LU_UrbFab_res','LU_Comm_res']] # convert to percentages
# remove high building density outliers (For Leipzig)
df=df.loc[df['UrbBuildDensity_res']<1e8,]  
df['UrbBuildDensity_res']=1e-6*df['UrbBuildDensity_res'] # convert from m3/km2 to m3/m2 

df.drop(columns='Mode',inplace=True)
# identify the feature columns
N_non_feature=7 # how many non-features are at the start of the df
cols=df.columns
newcols=(df.columns[:N_non_feature].tolist()) + ('FeatureM' +'_'+ cols[N_non_feature:-1]).tolist() + (df.columns[-1:].tolist())
# change column names
df.set_axis(newcols,axis=1,inplace=True)
df = df.reset_index(drop=True)
df.dropna(inplace=True)
df0=df.copy()

# convert  all categorical variables to dummies
df_Cat=df.select_dtypes('object')[[col for col in df.select_dtypes('object').columns if "FeatureM" in col]]
for col in df_Cat:
    dum=pd.get_dummies(df[[col]])
    df = pd.concat([df, dum], axis = 1)
    # remove the original categorical columns
df.drop(df_Cat.columns.tolist(),axis=1,inplace=True)
# HPO with full dataset, grouping by individual person
target = 'Mode_num'
N=len(df)

# Define the parameter space to be considered
PS = {"learning_rate": [0.1 ,0.2,0.3], 
                "n_estimators": [100, 250,400],
                "max_depth":[4, 5]}

X=df[[col for col in df.columns if "FeatureM" in col]]
y = df[target]

tf=(X < 0).all(0)
print(len(tf[tf]),' columns with value below zero')
if len(tf[tf])>0:
    print(tf[tf].index.values)
    raise Exception("Some columns have values below zero")

gr=df['HH_PNR']
groups=gr
gkf = list(GroupKFold(n_splits=9).split(X,y,groups)) # i found somewhere online that i had to define the cv splitter as a list, can't find the source at the moment.


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0  columns with value below zero


In [6]:
city='All'


In [7]:
form_str="Mode_num ~ FeatureM_Trip_Time + FeatureM_Season + FeatureM_Trip_Purpose_Agg + FeatureM_Sex + FeatureM_Age + FeatureM_Trip_Distance + FeatureM_CarOwnershipHH + FeatureM_HHSize + FeatureM_Occupation +  FeatureM_Education  + FeatureM_UrbPopDensity_res +  FeatureM_DistSubcenter_res + FeatureM_DistCenter_res_pc +  FeatureM_UrbBuildDensity_res +  FeatureM_IntersecDensity_res +  FeatureM_street_length_res +FeatureM_bike_lane_share_res +  FeatureM_LU_UrbFab_res + FeatureM_LU_Comm_res + FeatureM_transit_accessibility_res + FeatureM_Country" # FeatureM_CarAvailable replaced by FeatureM_CarOwnershipHH


In [8]:
fp='../outputs/ML_Results/'+city+'_HPO_mode_common_new_summary.csv'
if os.path.isfile(fp):
    print('HPs already identified')
    HPO_summary=pd.read_csv(fp)
    n_parameter_all = HPO_summary['N_est'][0]
    lr_parameter_all = HPO_summary['LR'][0]
    md_parameter_all = HPO_summary['MD'][0]
else:
    # define grid search cross validator
    tuning_all = GridSearchCV(estimator=XGBClassifier(verbosity=0,use_label_encoder=False), param_grid=PS, cv=gkf, scoring="f1_weighted",return_train_score=True)
    tuning_all.fit(X,y)

    print('best hyper-parameters identified by HPO')
    print(tuning_all.best_params_)
    print('model score with best hyper-paramteres')
    print(tuning_all.best_score_)
    cv_res_all=tuning_all.cv_results_

    n_parameter_all = tuning_all.best_params_['n_estimators']
    lr_parameter_all = tuning_all.best_params_['learning_rate']
    md_parameter_all = tuning_all.best_params_['max_depth']

    # save results of HPO
    r8=['gkf_gridSearch','full','9splits_hhperGroups',tuning_all.best_params_['learning_rate'],tuning_all.best_params_['max_depth'],tuning_all.best_params_['n_estimators'],round(tuning_all.best_score_,3),round(cv_res_all['std_test_score'][tuning_all.best_index_],3),N] #
    # also include other results lists here if HPO is done for more than one cv type or sample
    HPO_summary=pd.DataFrame([r8],columns=['CV_Type','Sample','CV_params','LR','MD','N_est','F1_best','SD_best','N_obs']) # the last element in this case is the sd of f1 scores in the fold which produced best results

# now redo the CV and calculate the SHAP values with the best HPs
cv = GroupKFold(n_splits=9)

y_predict = pd.DataFrame()
y_predict2 = pd.DataFrame()
y_test = pd.DataFrame()
y_test2 = pd.DataFrame()

shap_values0= pd.DataFrame()
shap_values1= pd.DataFrame()
shap_values2= pd.DataFrame()
shap_values3= pd.DataFrame()

summ_table_list=[]

model = XGBClassifier(
    max_depth=md_parameter_all, 
    n_estimators=n_parameter_all, 
    learning_rate=lr_parameter_all)

# model2 = LogisticRegression()
writer = pd.ExcelWriter('../outputs/ML_Results/mode_MNLR_new/'  + city + '.xlsx', engine='openpyxl')
for train_idx, test_idx in cv.split(X,groups=gr): # select here 
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test_fold = y.iloc[train_idx], y.iloc[test_idx]
    df_train, df_test = df0.iloc[train_idx], df0.iloc[test_idx]
    #id=test_idx[0].astype(str)
    id=datetime.now().strftime("%S%f")
    print('id',id)

    # train & predict
    model.fit(X_train, y_train, verbose=False, eval_set=[(X_train, y_train), (X_test, y_test_fold)])
    y_predict_fold = pd.Series(model.predict(X_test), index=X_test.index)

    # explain
    explainer = shap.TreeExplainer(model)
    shap_values_fold = explainer.shap_values(X_test)

    shap_values_fold0=shap_values_fold[0]
    shap_values_fold1=shap_values_fold[1]
    shap_values_fold2=shap_values_fold[2]
    shap_values_fold3=shap_values_fold[3]
    
    shap_values_fold0 = pd.DataFrame(shap_values_fold0, index=X_test.index, columns=X.columns)
    shap_values_fold1 = pd.DataFrame(shap_values_fold1, index=X_test.index, columns=X.columns)
    shap_values_fold2 = pd.DataFrame(shap_values_fold2, index=X_test.index, columns=X.columns)
    shap_values_fold3 = pd.DataFrame(shap_values_fold3, index=X_test.index, columns=X.columns)    

    y_predict = pd.concat([y_predict, y_predict_fold], axis=0)
    y_test = pd.concat([y_test, y_test_fold], axis=0)

    shap_values0 = pd.concat([shap_values0, shap_values_fold0], axis=0)
    shap_values1 = pd.concat([shap_values1, shap_values_fold1], axis=0)
    shap_values2 = pd.concat([shap_values2, shap_values_fold2], axis=0)
    shap_values3 = pd.concat([shap_values3, shap_values_fold3], axis=0)
    
    #model2.fit(X_train, y_train)
    # y_predict_fold2 = pd.Series(model2.predict(X_test), index=X_test.index)
    # y_predict2 = pd.concat([y_predict2, y_predict_fold2], axis=0)
    
    # X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    # y_train, y_test_fold = y.iloc[train_idx], y.iloc[test_idx]
    # df_train, df_test = df.iloc[train_idx], df.iloc[test_idx]
    y_test_fold2=df_test['Mode_num']

    try:
        log_reg = smf.mnlogit(form_str, data=df_train).fit()
        yhat=np.asarray(log_reg.predict(df_test.drop(columns='Mode_num'))).argmax(1)
        y_predict_fold2 = pd.Series(yhat, index=df_test.index)
        y_predict2 = pd.concat([y_predict2, y_predict_fold2], axis=0)
        y_test2 = pd.concat([y_test2, y_test_fold2], axis=0)

        coeff=log_reg.params.reset_index()
        coeff.rename(columns={'index':'param',0:'bike',1:'walk',2:'transit'},inplace=True)
        coeff.head()
        pval=log_reg.pvalues.reset_index()
        pval.rename(columns={'index':'param',0:'bike_p',1:'walk_p',2:'transit_p'},inplace=True)
        pval.head()
        summ_table=pd.concat([coeff,pval.loc[:,['bike_p','walk_p','transit_p']]],axis=1)
        summ_table['param']=summ_table['param'].str.replace('FeatureM_','')

        st_list_fold=[summ_table.drop(columns='param').to_numpy()]
        summ_table_list.append(st_list_fold)
        #summ_table.head()

        summ_table.to_excel(writer, sheet_name='summ' + id,index=False)
    except: 
        print('Logit Model Error')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

mdarray=np.array(summ_table_list).squeeze()
means=np.nanmean(mdarray,axis=0)
means_df=pd.DataFrame(data=np.hstack((np.reshape(summ_table['param'].to_numpy(),(len(summ_table),1)),means)),columns=summ_table.columns.values)
means_df.to_csv('../outputs/ML_Results/mode_MNLR_new/'  + city + '_mean.csv',index=False)

y_test = y_test.squeeze(axis=1)
y_test2 = y_test2.squeeze(axis=1)
y_predict = y_predict.squeeze(axis=1)
y_predict2 = y_predict2.squeeze(axis=1)
f1_model_ML = metrics.f1_score(y_test, y_predict, average='weighted')
f1_model_LR = metrics.f1_score(y_test2, y_predict2, average='weighted')
print('Model f1: ' + city)
print(f1_model_ML)

HPO_summary['F1_full_ML']=f1_model_ML
HPO_summary['F1_full_LR']=f1_model_LR
HPO_summary['City']=city
HPO_summary.to_csv('../outputs/ML_Results/' + city + '_HPO_mode_common_new_summary.csv',index=False)

# optionally here, check which variables are more important than random noise, then downselect X to those variables, and go back to HPO (or CV) and run once more from there.

shap_valueslist=[shap_values0.sort_index().to_numpy(),shap_values1.sort_index().to_numpy(),shap_values2.sort_index().to_numpy(),shap_values3.sort_index().to_numpy()]
X_disp=[re.sub('FeatureM_','', x) for x in X.sort_index().columns]
shap_values_abs=abs(shap_valueslist[0])+abs(shap_valueslist[1])+abs(shap_valueslist[2])+abs(shap_valueslist[3])
shap_sum = shap_values_abs.mean(axis=0)
importance_df = pd.DataFrame([X_disp, shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)

with open('../outputs/ML_Results/shap/mode_common_new/' + city + '_importance.pkl', 'wb') as g:
    pickle.dump(importance_df, g)

# shap_mode = shap_disp(shap_valueslist,X.sort_index(), 'Mode Choice ', city)

# save shap_values, to enable later re-creation and editing of shap plots
with open('../outputs/ML_Results/shap/mode_common_new/' + city + '.pkl', 'wb') as f:
    pickle.dump(shap_valueslist, f)

shap.summary_plot(shap_valueslist, X.sort_index(),feature_names=X_disp,max_display=16,  class_names=['car','bike','foot','trans'],class_inds='original', show=False)
plt.title('Overall Feature Influence Mode Choice ' + city)
plt.savefig('../outputs/ML_Results/result_figures/mode_common_new/' + city + '_mode_FI.png',facecolor='w',dpi=65,bbox_inches='tight')
plt.close() 


cm = metrics.confusion_matrix(y_test, y_predict,normalize='true')
# note the high confusion between bike/foot trips, and the high number of transit trips labelled as bike or car, and the high number of bike trips labelled as foot or car
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.array(['car','bike','foot','trans']))
disp.plot()
plt.title('Confusion matrix, mode choice, ' + city + '. F1: ' + str(round(f1_model_ML,3)))
plt.savefig('../outputs/ML_Results/result_figures/mode_common_new/' + city + '_mode_CM.png',facecolor='w',dpi=65,bbox_inches='tight')
plt.close() 

col_dict= {'DistCenter_res_pc':'Dist. to city center','DistSubcenter_res':'Dist. to subenter', 'UrbPopDensity_res':'Population density','UrbBuildDensity_res':'Built-up density','ParkingAvailable_Dest':'Parking available',
    'IntersecDensity_res':'Intersection density','LU_Comm_res':'Commercial area','LU_UrbFab_res':'Urban Fabric area','street_length_res':'Avg. street length','bike_lane_share_res':'Cycle lanes',
    'Trip_Purpose_Agg_Home↔Work':'Commute trip', 'Trip_Purpose_Agg_Home↔Companion':'Companion trip', 'TravelAlone':'Solo trip','Trip_Purpose_Agg_Home↔Leisure':'Leisure trip','Trip_Purpose_Agg_Home↔Shopping':'Shopping trip','Trip_Purpose_Agg_Home↔School':'School trip',
    'Trip_Time_Evening':'Evening trip','Trip_Time_AM_Rush':'Morning rush-hour','Trip_Time_PM Rush':'Evening rush-hour','transit_accessibility_res':'Transit accessibility',
    'Country_France':'France','Country_Germany':'Germany','Country_Austria':'Austria','Country_Spain':'Spain',
    'Season_Winter':'Winter season','MeanTime2Transit_res':'Time to transit', #'diff':'Elevation_diff',
    'Trip_Distance':'Trip distance','CarOwnershipHH':'Car ownership','Occupation_Student_School':'School Student',
    'Age':'Age','Sex':'Sex','HHSize':'Household size','IncomeDescriptiveNumeric':'Income','IncomeDetailed_Numeric':'Income',
    'Education_University':'University education', 'Occupation_Employed_FullTime':'Employed'}

X_lab=[*map(col_dict.get, X_disp)]

shap_values=shap_valueslist

if city not in ['Berlin','Paris','Madrid','Wien','France_other','Germany_other']:
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(11,8))
    ax1 = plt.subplot(221)
    shap.summary_plot(shap_values[2], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Foot Trips ' + city)
    plt.xlabel("SHAP value", size=11)
    ax2 = plt.subplot(222)
    shap.summary_plot(shap_values[1], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Bike Trips ' + city)
    plt.xlabel("SHAP value", size=11)
    ax3 = plt.subplot(223)
    shap.summary_plot(shap_values[0], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Car Trips ' + city)
    plt.xlabel("SHAP value", size=11)
    ax3 = plt.subplot(224)
    shap.summary_plot(shap_values[3], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Transit Trips ' + city)
    plt.xlabel("SHAP value", size=11)
    plt.savefig('../outputs/ML_Results/result_figures/mode_common_new/' + city + '_FI_all.png',facecolor='w',dpi=65,bbox_inches='tight')
    plt.close() 


X_lab=[*map(col_dict.get, X_disp)]
if city == 'Berlin': let='a'
if city == 'Paris': let='b'
if city == 'Madrid': let='c'
if city == 'Wien': let='d'
if city == 'Germany_other': let='e'
if city == 'France_other': let='f'
if city == 'All': let='g'

if city in ['Berlin','Paris','Madrid','Wien','France_other','Germany_other','All']:
    fig, axes = plt.subplots(figsize=(5.5,4))
    shap.summary_plot(shap_values[0], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title(let + ') ' + city,size=14)
    plt.xlabel("SHAP (probability of car mode choice)", size=11)
    plt.savefig('../outputs/ML_Results/result_figures/mode_common_new/' + city + '_FI_car.png',facecolor='w',dpi=65,bbox_inches='tight')
    plt.close() 

    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(11,8))
    ax1 = plt.subplot(221)
    shap.summary_plot(shap_values[2], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title(let + ') ' + 'Main Feature Influences for Foot Trips ' + city.replace('_',', '))
    plt.xlabel("SHAP (probability of foot mode)", size=11)
    ax2 = plt.subplot(222)
    shap.summary_plot(shap_values[1], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Bike Trips ' + city.replace('_',', '))
    #plt.title(city.replace('_',', '))
    plt.xlabel("SHAP (probability of bike mode)", size=11)
    ax3 = plt.subplot(223)
    shap.summary_plot(shap_values[0], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Car Trips ' + city.replace('_',', '))
    #plt.title(city.replace('_',', '))
    plt.xlabel("SHAP (probability of car mode)", size=11)
    ax3 = plt.subplot(224)
    shap.summary_plot(shap_values[3], X, feature_names=X_lab, max_display=8, plot_size=None, show=False)
    plt.title('Main Feature Influences for Transit Trips ' + city.replace('_',', '))
    #plt.title(city.replace('_',', '))
    plt.xlabel("SHAP (probability of transit mode)", size=11)

    plt.savefig('../outputs/ML_Results/result_figures/mode_common_new/' + city + '_FI_all.png',facecolor='w',dpi=65,bbox_inches='tight')
    plt.close() 

HPs already identified


id 25108457


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[15:52:25] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


...

Optimization terminated successfully.
         Current function value: 0.735109
         Iterations 9
id 18645416


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


[16:23:19] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


Optimization terminated successfully.
         Current function value: 0.734982
         Iterations 9
Model f1: All
0.7577787121581939
